In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import pearsonr

## Load Data 
file_path = "complete.csv"
data = pd.read_csv("complete.csv", on_bad_lines="skip") # there are some bad lines in the csv file so I skip them

##  Data Cleaning-up/Manipulation
data = data.dropna()  # drop rows with missing values
data = data.drop_duplicates()  # drop duplicate rows 
data = data.reset_index(drop=True)  # reset index after dropping rows
data_us = data[data['country'].str.lower() == 'us'] # drop rows not from the US
data_us = data_us.dropna(subset=['state']) # drop rows with missing state info just in case
# I dropped the rows with missing values and duplicates, because I want to keep the data clean and accurate for analysis.
# Also, I dropped the rows not from the US and the rows with missing state info, because I want to focus on the US data and make sure I have complete information for each state.


print(len(data_us))   # Number of rows BEFORE (check)

# date transformation
data_us['datetime'] = pd.to_datetime(data_us['datetime'], errors='coerce')
data_us['month'] = data_us['datetime'].dt.month.astype('Int64')
data_us['day'] = data_us['datetime'].dt.day.astype('Int64')
data_us['year'] = data_us['datetime'].dt.year.astype('Int64')

print(len(data_us))   # Number of rows AFTER  (check)

year_counts = data_us['year'].value_counts().sort_index()
year_counts.plot(kind='bar', figsize=(12, 5))
plt.xlabel('Year')
plt.ylabel('Number of Sightings')
plt.title('UFO Sightings per Year (US Only)')
plt.tight_layout()
plt.show()

# dictionary to map 2-letter codes to full lowercase state names
state_abbrev_to_full = {
    'al': 'alabama', 'ak': 'alaska', 'az': 'arizona', 'ar': 'arkansas',
    'ca': 'california', 'co': 'colorado', 'ct': 'connecticut', 'de': 'delaware',
    'fl': 'florida', 'ga': 'georgia', 'hi': 'hawaii', 'id': 'idaho',
    'il': 'illinois', 'in': 'indiana', 'ia': 'iowa', 'ks': 'kansas',
    'ky': 'kentucky', 'la': 'louisiana', 'me': 'maine', 'md': 'maryland',
    'ma': 'massachusetts', 'mi': 'michigan', 'mn': 'minnesota', 'ms': 'mississippi',
    'mo': 'missouri', 'mt': 'montana', 'ne': 'nebraska', 'nv': 'nevada',
    'nh': 'new hampshire', 'nj': 'new jersey', 'nm': 'new mexico', 'ny': 'new york',
    'nc': 'north carolina', 'nd': 'north dakota', 'oh': 'ohio', 'ok': 'oklahoma',
    'or': 'oregon', 'pa': 'pennsylvania', 'ri': 'rhode island', 'sc': 'south carolina',
    'sd': 'south dakota', 'tn': 'tennessee', 'tx': 'texas', 'ut': 'utah',
    'vt': 'vermont', 'va': 'virginia', 'wa': 'washington', 'wv': 'west virginia',
    'wi': 'wisconsin', 'wy': 'wyoming', 'dc': 'district of columbia',
    'pr': 'puerto rico'  # puerto rico is not a state but is included in the UFO dataset
}

# map state codes to full names and Visualize
data_us['state'] = data_us['state'].str.lower()  # ensure lowercase for mapping
data_us['state_full'] = data_us['state'].map(state_abbrev_to_full)
state_counts_full = data_us['state_full'].value_counts().sort_values(ascending=False)
state_counts_full.plot(kind='bar', figsize=(16, 6))
plt.xlabel('State (Full Name)')
plt.ylabel('Number of Sightings')
plt.title('UFO Sightings by U.S. State (Full Names)')
plt.tight_layout()
plt.show()


## Data Cleaning-up/Manipulation/loading (continued)
death_d = pd.read_csv("us_death_data.csv", on_bad_lines="skip") # there are some bad lines in the csv file so I skip them
alcohol_d = pd.read_csv("alcohol_consumption_2013.csv", on_bad_lines="skip")
pop_d = pd.read_csv("us_state_population_2013(july).csv", on_bad_lines="skip")
alcohol_d['State'] = alcohol_d['State'].str.lower()
death_d["State"] = death_d["State"].str.lower()

# filtering data_us for only year 2013 beacause the alcohol consumption data is only from 2013
data_us_2013 = data_us[data_us['year'] == 2013].copy()
#  clean both sides (lowercase + strip spaces)
data_us_2013['state_full'] = data_us_2013['state_full'].str.lower().str.strip()
alcohol_d['State'] = alcohol_d['State'].str.lower().str.strip()
# merging
merged_df = pd.merge(data_us_2013, alcohol_d, left_on='state_full', right_on='State', how='inner')
# droping duplicate 'State' column
merged_df = merged_df.drop(columns=['State'])
# check
print(merged_df.shape)
merged_df.head()


#  making sure state names are clean
merged_df['state_full'] = merged_df['state_full'].str.lower().str.strip()
pop_d['State'] = pop_d['State'].str.lower().str.strip()
#  Count UFO sightings per state (only for 2013 sightings)
sightings_per_state_2013 = merged_df['state_full'].value_counts().reset_index()
sightings_per_state_2013.columns = ['state_full', 'UFO Sightings']
#  merge with population data
merged_pop_2013 = pd.merge(sightings_per_state_2013, pop_d, left_on='state_full', right_on='State', how='left')
# Calculate Sightings per 100,000 people
merged_pop_2013['Sightings per 100k'] = (merged_pop_2013['UFO Sightings'] / merged_pop_2013['Population']) * 100000
#  Clean merged_df first (drop any old 'Sightings per 100k' if exist)
merged_df = merged_df.drop(columns=[col for col in merged_df.columns if 'Sightings per 100k' in col], errors='ignore')
#  Merge back to merged_df
merged_df = pd.merge(merged_df, merged_pop_2013[['state_full', 'Sightings per 100k']], on='state_full', how='left')
# final check
merged_df.head()


## Data Visualization
# 1. UFO Sightings per 100k vs All Beverages Consumption
plt.figure(figsize=(14, 9))
plt.scatter(merged_df['All Beverages'], merged_df['Sightings per 100k'])
plt.title('UFO Sightings per 100k vs All Alcohol Consumption (2013)')
plt.xlabel('All Beverages Consumption per Capita (Gallons)')
plt.ylabel('UFO Sightings per 100,000 People')
m, b = np.polyfit(merged_df['All Beverages'], merged_df['Sightings per 100k'], 1)
plt.plot(merged_df['All Beverages'], m * merged_df['All Beverages'] + b, color='red')
r_value, p_value = pearsonr(merged_df['All Beverages'], merged_df['Sightings per 100k'])
plt.figtext(0.5, -0.05, f"Correlation coefficient (r) = {r_value:.4f}", ha='center', fontsize=12)

plt.grid(True)
plt.tight_layout()
plt.show()




# 2. UFO Sightings per 100k vs Wine Consumption
plt.figure(figsize=(14, 9))
plt.scatter(merged_df['Wine'], merged_df['Sightings per 100k'])
plt.title('UFO Sightings per 100k vs Wine Consumption (2013)')
plt.xlabel('Wine Consumption per Capita (Gallons)')
plt.ylabel('UFO Sightings per 100,000 People')
m, b = np.polyfit(merged_df['Wine'], merged_df['Sightings per 100k'], 1)
plt.plot(merged_df['Wine'], m * merged_df['Wine'] + b, color='red')
r_value, p_value = pearsonr(merged_df['Wine'], merged_df['Sightings per 100k'])
plt.figtext(0.5, -0.05, f"Correlation coefficient (r) = {r_value:.4f}", ha='center', fontsize=12)

plt.grid(True)
plt.tight_layout()
plt.show()



# 3. UFO Sightings per 100k vs Beer Consumption
plt.figure(figsize=(14, 9))
plt.scatter(merged_df['Beer'], merged_df['Sightings per 100k'])
plt.title('UFO Sightings per 100k vs Beer Consumption (2013)')
plt.xlabel('Beer Consumption per Capita (Gallons)')
plt.ylabel('UFO Sightings per 100,000 People')
m, b = np.polyfit(merged_df['Beer'], merged_df['Sightings per 100k'], 1)
plt.plot(merged_df['Beer'], m * merged_df['Beer'] + b, color='red')
r = np.corrcoef(merged_df['Beer'], merged_df['Sightings per 100k'])[0, 1]
plt.figtext(0.5, -0.05, f'Correlation coefficient (r) = {r:.4f}', ha='center', fontsize=12)

plt.grid(True)
plt.tight_layout()
plt.show()




# 4. UFO Sightings per 100k vs Spirits Consumption
plt.figure(figsize=(14, 9))
plt.scatter(merged_df['Spirits'], merged_df['Sightings per 100k'])
plt.title('UFO Sightings per 100k vs Spirits Consumption (2013)')
plt.xlabel('Spirits Consumption per Capita (Gallons)')
plt.ylabel('UFO Sightings per 100,000 People')
m, b = np.polyfit(merged_df['Spirits'], merged_df['Sightings per 100k'], 1)
plt.plot(merged_df['Spirits'], m * merged_df['Spirits'] + b, color='red')
r_value, p_value = pearsonr(merged_df['Spirits'], merged_df['Sightings per 100k'])
plt.figtext(0.5, -0.05, f"Correlation coefficient (r) = {r_value:.4f}", ha='center', fontsize=12)

plt.grid(True)
plt.tight_layout()
plt.show()

# 5. Correlation Heatmap
heatmap_data = merged_df[['Beer', 'Wine', 'Spirits', 'All Beverages', 'Sightings per 100k']]
corr_matrix = heatmap_data.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Correlation Heatmap: Alcohol Consumption vs UFO Sightings per 100k (2013)', fontsize=14)
plt.tight_layout()
plt.show()



# 6. UFO Sightings by day number (additional analysis)
day_number_counts = data_us['day'].value_counts().sort_index() # count the number of sightings for each day of the month	
day_number_counts.plot(kind='bar', figsize=(12, 5))
plt.xlabel('Day of the Month')
plt.ylabel('Number of Sightings')
plt.title('UFO Sightings by Day Number (US Only)')
plt.xticks(range(0, 31), [str(i+1) for i in range(31)]) 
plt.tight_layout()
plt.show()